## Extracción, transformación y carga (ETL) ##

- En primer lugar voy a importar las librerias que voy a utilizar en este proceso

In [21]:
import pandas as pd
import numpy as np
import locale 
import ast 
from pandas import json_normalize 

- Luego voy a leer el primer archivo con el que voy a comenzar a trabajar
1) movies_dataset.csv

In [22]:
#1)
df = pd.read_csv ("/Users/romina/Desktop/Datasets/movies_dataset.csv")

/var/folders/n3/ppx4r_992xxbdk4s0ht21__m0000gn/T/ipykernel_5594/847602983.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv ("/Users/romina/Desktop/Datasets/movies_dataset.csv")


- A continuación, voy a proceder a realizar el desanidado de la base de datos movies_dataset.csv, visualizados en listas y en diccionarios para ver el contenido de cada uno de ellos

In [23]:
#Defino la función desanidadolista la cual devuelve una lista de nombres
def desanidadolista(obj): #Defino la función desanidadolista la cual devuelve una lista de nombres
    if isinstance(obj, str) and '{' in obj: #La función revisa si el objeto es una cadena de texto y si contiene el carácter {}
        lista = [] #Creo una lista vacía
        for i in ast.literal_eval(obj): #Comienza a recorrer cada elemento del objeto
            lista.append(i['name']) #Se agrega el valor asociado a la clave name de un diccionario i al final de la lista. Cada vez que se ejecuta esta línea, se agrega un nuevo elemento a la lista
        return lista


#Defino la función desanidadodiccionario a modo de visualizar lo que contiene
def desanidadodiccionario(obj): #Defino la función desanidadodiccionario para ver que es lo que contiene
    if isinstance(obj, str) and '{' in obj:
        diccionario = ast.literal_eval(obj)
        return diccionario ['name']

#Aplico las funciones anteriormente mencionadas a cada una de las columnas que así lo requieren y sobreescribo el df
df["genres"] = df["genres"].apply(desanidadolista) 
df["belongs_to_collection"] = df["belongs_to_collection"].apply(desanidadodiccionario)
df["production_companies"]  = df["production_companies"].apply(desanidadolista)
df["production_countries"]  = df["production_countries"].apply(desanidadolista)
df["spoken_languages"] = df["spoken_languages"].apply(desanidadolista)

In [24]:
#Observo que contiene la base de datos con las columnas correspondientes desanidadas
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [25]:
#Miro el tamaño del DF
df.shape

(45466, 24)

- Relleno los valores nulos de los campos revenue y budget con 0

In [26]:
df["revenue"].fillna("0",inplace=True)
df["budget"].fillna("0",inplace=True)

In [27]:
#Observo ambos campos
df.isnull().sum()

adult                        0
belongs_to_collection    40975
budget                       0
genres                    2442
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      0
runtime                    263
spoken_languages          3835
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [28]:
#Obtengo información resumida del df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 43024 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   33585 non-null  object 
 13  production_countries   39178 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

- Convierto el campo release_date en datetime y le el formato correspondiente. Luego realizo la eliminación de nulos a partir de la creación de una máscara

In [29]:
#Paso las fechas a formato año-mes-día | Establezco valores incorrectos o no reconocidos como nulos
df ['release_date'] = pd.to_datetime(df["release_date"], format='%Y-%m-%d', errors = 'coerce')

#Creo una máscara para identificar las filas del df que contiene valores nulos
mascara = df[df["release_date"].isnull()] 

#Elimino filas del df con valores nulos | Agrega 0 a los valores nulos que elimina
df = df.drop(labels= mascara.index, axis=0)

- Convierto los campos revenue y budget en numéricos y genero una nueva columna llamada return

In [30]:
#Paso los valores a numéricos
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")

#Creo una nueva columna 'return' que calcula el cociente entre revenue y budget
df["return"] = df["revenue"] / df["budget"]

- Reemplazo los valores +infinito -infinito por nulos
- Como paso siguiente convierto los nulos en 0 y finalmente redondeo los números

In [31]:
#Reemplazo los valores infinitos de la columna return con NaN
df["return"] = df["return"].replace([np.inf, -np.inf], np.nan)

#Relleno los valores faltantes de la columna return con 0
df["return"] = df["return"].fillna(0)
df["return"] = df["return"].round(0)

- Procedo a eliminar las colmnas que no voy a utilizar

In [32]:
df.drop("video",axis="columns",inplace=True)
df.drop("imdb_id",axis="columns",inplace=True)
df.drop("adult",axis="columns",inplace=True)
df.drop("original_title",axis="columns",inplace=True)
df.drop("poster_path",axis="columns",inplace=True)
df.drop("homepage",axis="columns",inplace=True)

- Genero 2 columnas nuevas con el mes y el día de la fecha de estreno de las películas en formato texto en español
- Luego creo una tercer columna llamada release_year, en la cual voy a extraer el año de la fecha de estreno de cada una de las películas

In [33]:
df["month_name"] = df["release_date"].dt.month_name(locale.setlocale(locale.LC_TIME, 'es_ES'))
df["day_name"] = df["release_date"].dt.day_name(locale.setlocale(locale.LC_TIME, 'es_ES'))
df["release_year"] = df["release_date"].dt.year

In [34]:
df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,month_name,day_name,release_year
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,NaN,Toy Story,7.7,5415.0,12.0,Octubre,Lunes,1995
1,None,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.0,Diciembre,Viernes,1995


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  int64         
 2   genres                 42992 non-null  object        
 3   id                     45376 non-null  object        
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  object        
 7   production_companies   33580 non-null  object        
 8   production_countries   39165 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       41608 non-null  object        
 13  s

- Como último paso luego de haber implementado las modificaciones y el ETL, voy a generar un nuevo csv con los datos limpios para el consumo de la API

In [36]:
df.to_csv("/Users/romina/Desktop/Datasets/peliculas.csv",index=False)

In [37]:
df.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,month_name,day_name,release_year
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,NaN,Toy Story,7.7,5415.0,12.0,Octubre,Lunes,1995
1,None,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.0,Diciembre,Viernes,1995


- Voy a proceder a leer el segundo dataset con el que voy a trabajar
2) Credits.csv

In [38]:
df = pd.read_csv ("/Users/romina/Desktop/Datasets/credits.csv")
df #Visualizo lo que contiene la base de datos

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


- A partir de la siguiente función voy a comenzar a desanidar los datos de la columna crew

In [39]:
df['crew'] = df['crew'].apply(lambda x: ast.literal_eval(x)) #Utilizo la función lambda con respecto a cada valor de la columna crew 
dfcrew = df[['id', 'crew']].explode('crew') #Creo un nuevo df. Luego utilizo el método explode para expandir las listas en filas separadas 
dfcrew

,id,crew
0,862,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep..."
0,862,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep..."
0,862,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep..."
0,862,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep..."
0,862,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep..."
...,...,...
45473,67758,"{'credit_id': '52fe4776c3a368484e0c8399', 'dep..."
45473,67758,"{'credit_id': '52fe4776c3a368484e0c839f', 'dep..."
45474,227506,"{'credit_id': '533bccebc3a36844cf0011a7', 'dep..."
45474,227506,"{'credit_id': '58ebbc26925141281908aa0a', 'dep..."


### Crew ###

In [40]:
df_crew_desanidado = json_normalize(dfcrew["crew"]) #Utilizo la función mencionada pra desanidar y normalizar los datos json
df_crew_desanidado #Imprimo el nuevo df con los datos desanidados

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
465080,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
465081,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
465082,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
465083,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


In [41]:
#Guardo mi df en un nuevo archivo csv
df_crew_desanidado.to_csv("/Users/romina/Desktop/Datasets/crew.csv")

In [42]:
df['cast'] = df['cast'].apply(lambda x: ast.literal_eval(x)) #Utilizo la función lambda con respecto a cada valor de la columna cast 
dfcast = df[['id', 'cast']].explode('cast') #Creo un nuevo df. Luego utilizo el método explode para expandir las listas en filas separadas 
dfcast

,id,cast
0,862,"{'cast_id': 14, 'character': 'Woody (voice)', ..."
0,862,"{'cast_id': 15, 'character': 'Buzz Lightyear (..."
0,862,"{'cast_id': 16, 'character': 'Mr. Potato Head ..."
0,862,"{'cast_id': 17, 'character': 'Slinky Dog (voic..."
0,862,"{'cast_id': 18, 'character': 'Rex (voice)', 'c..."
...,...,...
45474,227506,"{'cast_id': 3, 'character': '', 'credit_id': '..."
45474,227506,"{'cast_id': 4, 'character': '', 'credit_id': '..."
45474,227506,"{'cast_id': 5, 'character': '', 'credit_id': '..."
45474,227506,"{'cast_id': 6, 'character': '', 'credit_id': '..."


### Cast ###

In [43]:
df_cast_desanidado = json_normalize(dfcast["cast"]) #Utilizo la función mencionada para desanidar y normalizar los datos json
df_cast_desanidado #Imprimo el nuevo df con los datos desanidados

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
564887,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [44]:
#Guardo mi df en un nuevo archivo csv
df_cast_desanidado.to_csv("/Users/romina/Desktop/Datasets/cast.csv") #Guardo mi df en un nuevo archivo csv

In [45]:
df_sorted = df_crew_desanidado.sort_values(by="id", ascending=False) #Ordenoo el df con la columna id en orden descendente
df_filtered = df_sorted[df_sorted["id"] == 1] #Filtro el df donde el valor de la columna id es = 1
df_filtered

,credit_id,department,gender,id,job,name,profile_path
69313,571ae3439251412e22000fdf,Production,2.0,1.0,Executive Producer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
40800,52fe431ec3a36847f803bca3,Writing,2.0,1.0,Screenplay,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
81591,52fe431fc3a36847f803bd8b,Directing,2.0,1.0,Director,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
4496,52fe420dc3a36847f8000437,Directing,2.0,1.0,Director,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
20069,52fe4216c3a36847f8002e93,Writing,2.0,1.0,Characters,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
...,...,...,...,...,...,...,...
331595,54625560c3a3680820000186,Production,2.0,1.0,Executive Producer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
17839,52fe431ec3a36847f803bbcb,Production,2.0,1.0,Executive Producer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
17840,52fe431ec3a36847f803bbc5,Writing,2.0,1.0,Story,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
81595,52fe431fc3a36847f803bda3,Writing,2.0,1.0,Screenplay,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg


In [46]:
df_crew_desanidado = df_crew_desanidado.drop_duplicates(subset=["job", "name","department"], keep="first") #Elimino las filas fuplicadas del df
df_crew_desanidado

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
465073,5862a509c3a36852d2000097,Art,0.0,1728582.0,Production Design,Dante Perez,None
465074,5862a51e9251412b870000a4,Sound,0.0,1051381.0,Music,Lav Diaz,/lWZ6mJCrPga6Jis1gbJYcpWpBZo.jpg
465076,5862a53a9251412b8a0000b7,Crew,0.0,1051381.0,Cinematography,Lav Diaz,/lWZ6mJCrPga6Jis1gbJYcpWpBZo.jpg
465079,52fe4776c3a368484e0c8393,Writing,2.0,549355.0,Screenplay,Jeffrey Goldenberg,/cX9qmOSujqnjIqDVS1jlNSpWLHx.jpg


In [47]:
df_sorted = df_crew_desanidado.sort_values(by="id", ascending=False) #Ordeno el df con la columna id en orden descendente 
df_filtered = df_sorted[df_sorted["id"] == 1] #Filtro el df donde el valor de la columna id es = 1
df_filtered

,credit_id,department,gender,id,job,name,profile_path
81592,52fe431fc3a36847f803bd91,Production,2.0,1.0,Producer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
17838,566e19f292514169e200d46f,Editing,2.0,1.0,Editor,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
4498,562e75309251414006009955,Writing,2.0,1.0,Writer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
20069,52fe4216c3a36847f8002e93,Writing,2.0,1.0,Characters,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
4496,52fe420dc3a36847f8000437,Directing,2.0,1.0,Director,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
133902,52fe431dc3a36847f803b791,Writing,2.0,1.0,Author,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
18235,52fe431ec3a36847f803bc0f,Writing,2.0,1.0,Screenplay,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
32522,588a748cc3a368409301c92a,Crew,2.0,1.0,Thanks,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
8426,52fe4447c3a368484e019f85,Writing,2.0,1.0,Story,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
4497,52fe420dc3a36847f800045b,Production,2.0,1.0,Executive Producer,George Lucas,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg


In [48]:
df_cast_desanidado

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
564887,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [49]:
df_sorted = df_cast_desanidado.sort_values(by="id", ascending=False) #Ordeno la columna id en orden descendente 
df_filtered = df_sorted[df_sorted["id"] == 1] #A partir de un filtrado visualizo solo aquellas donde el valor de id es = 1
df_filtered

,cast_id,character,credit_id,gender,id,name,order,profile_path
6894,5.0,Disappointed Man,52fe4235c3a36847f800c2f7,2.0,1.0,George Lucas,21.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
161791,78.0,Baron Papanoida,584a4128c3a368141f01b620,2.0,1.0,George Lucas,24.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
303226,1.0,Himself,52fe48799251416c9108db99,2.0,1.0,George Lucas,0.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
273577,1010.0,Himself (archive footage),54a69d09925141236b00e139,2.0,1.0,George Lucas,9.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
240151,11.0,Himself,56757ade92514179d4002e15,2.0,1.0,George Lucas,7.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
174078,15.0,Himself,52fe48dac3a36847f817cdcb,2.0,1.0,George Lucas,10.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
423007,19.0,Himself (voice),52fe4624c3a36847f80ef089,2.0,1.0,George Lucas,9.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
290920,22.0,Himself,52fe4ad2c3a36847f81e3925,2.0,1.0,George Lucas,8.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
58661,51.0,Man Kissing on Bridge (uncredited),5781e003c3a3681f4d000f21,2.0,1.0,George Lucas,27.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
430980,22.0,Himself,52fe49939251416c750cc87b,2.0,1.0,George Lucas,11.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg


In [50]:
df_cast_desanidado = df_cast_desanidado.drop_duplicates(subset=[ "name","character"], keep="first") #Elimino los valores duplicados 
df_cast_desanidado #Imprimo el df actualizado

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
564886,2.0,,52fe4ea59251416c7515d7d5,2.0,544742.0,Iwan Mosschuchin,0.0,None
564887,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None


In [51]:
df_sorted = df_cast_desanidado.sort_values(by="id", ascending=False) #Ordeno el df por la columna id en orden descendente y asigno un nuevo df 
df_filtered = df_sorted[df_sorted["id"] == 1] #Filtro el df para seleccionar las filas donde el valor de la columna id es = 1
df_filtered

,cast_id,character,credit_id,gender,id,name,order,profile_path
147057,28.0,Himself,52fe43cac3a36847f806fc33,2.0,1.0,George Lucas,3.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
273577,1010.0,Himself (archive footage),54a69d09925141236b00e139,2.0,1.0,George Lucas,9.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
161791,78.0,Baron Papanoida,584a4128c3a368141f01b620,2.0,1.0,George Lucas,24.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
58661,51.0,Man Kissing on Bridge (uncredited),5781e003c3a3681f4d000f21,2.0,1.0,George Lucas,27.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
423007,19.0,Himself (voice),52fe4624c3a36847f80ef089,2.0,1.0,George Lucas,9.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
35421,88.0,Missionary (uncredited),5895b352c3a36822e0004ad5,2.0,1.0,George Lucas,66.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
6894,5.0,Disappointed Man,52fe4235c3a36847f800c2f7,2.0,1.0,George Lucas,21.0,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg


### Una vez finalizado mi ETL, voy a continuar realizando el EDA de mi proyecto... ###